# 🎹 Piano Transcription Model Training

**Goal**: Train a CRNN model that transcribes piano audio → MIDI with onset, frame, and velocity.

**Architecture**: Mel → Conv2D → BiLSTM → onset/frame/velocity heads (88 piano keys)

**Dataset**: MAESTRO v3.0.0 (198 hours of Yamaha Disklavier piano with ~3ms MIDI alignment)

**Why MAESTRO?**
- Gold standard for piano AMT (Automatic Music Transcription)
- ~3ms alignment between audio and MIDI from Disklavier hardware capture
- 198 hours across 1,276 performances from International Piano-e-Competition
- Full velocity, pedal (sustain, sostenuto, una corda) annotations
- Used by Google Magenta, ByteDance, Sony AI for SOTA piano models

**Output**: `best_piano_model.pt` — drop into `backend/models/pretrained/`

---
**Estimated Training Time**:
- A100: ~6-12 hours (50 epochs)
- T4: ~18-36 hours

**Disk Requirements**: ~120 GB (MAESTRO WAV + MIDI)

## 0. GPU Check & Setup

In [ ]:
import torch
import os

# GPU check
if torch.cuda.is_available():
    gpu_name = torch.cuda.get_device_name(0)
    gpu_mem = torch.cuda.get_device_properties(0).total_memory / 1e9
    print(f'GPU: {gpu_name} ({gpu_mem:.1f} GB)')
else:
    print('WARNING: No GPU detected! Training will be very slow.')
    print('Go to Runtime > Change runtime type > GPU (A100 preferred)')

# Mount Google Drive for checkpoint saving
from google.colab import drive
drive.mount('/content/drive')

# Create output directory on Drive
DRIVE_OUTPUT = '/content/drive/MyDrive/piano_model_results'
os.makedirs(DRIVE_OUTPUT, exist_ok=True)
print(f'Checkpoints will save to: {DRIVE_OUTPUT}')

In [ ]:
# Install dependencies
!pip install -q librosa pretty_midi tqdm soundfile

## 1. Download MAESTRO v3.0.0

~120 GB WAV dataset. MIDI-only is 58 MB but we need the audio too.

In [ ]:
import subprocess
from pathlib import Path
import json
import shutil as sh

MAESTRO_DIR = Path('/content/maestro-v3.0.0')

# Check if already downloaded (reconnect-safe: need 1200+ WAV files for full dataset)
if MAESTRO_DIR.exists() and len(list(MAESTRO_DIR.glob('**/*.wav'))) > 1200:
    print(f'MAESTRO already downloaded: {len(list(MAESTRO_DIR.glob("**/*.wav")))} WAV files')
else:
    print('Downloading MAESTRO v3.0.0 from Google Storage...')
    print('This is ~101 GB — will take 15-30 minutes on Colab')
    print()

    ZIP_PATH = '/content/maestro-v3.0.0.zip'

    # Check available disk space
    total, used, free = sh.disk_usage('/content')
    free_gb = free / 1e9
    print(f'Disk space: {free_gb:.1f} GB free')

    if free_gb < 210:
        # Not enough to hold zip + extracted simultaneously
        # Use gsutil to download files directly (no zip needed)
        print('Using gsutil to download directly (avoids zip disk overflow)...')
        print('This downloads files individually — slower but disk-safe.')

        !mkdir -p /content/maestro-v3.0.0
        !gsutil -m cp -r 'gs://magentadata/datasets/maestro/v3.0.0/maestro-v3.0.0/*' /content/maestro-v3.0.0/

    else:
        # Plenty of space — use the zip (faster)
        !wget -q --show-progress -O {ZIP_PATH} \
            'https://storage.googleapis.com/magentadata/datasets/maestro/v3.0.0/maestro-v3.0.0.zip'

        print('\nExtracting...')
        !cd /content && unzip -q maestro-v3.0.0.zip

        # Delete zip immediately to free ~101 GB
        zip_path = Path(ZIP_PATH)
        if zip_path.exists():
            zip_path.unlink()
            print('Deleted zip to free disk space')

# Verify
wav_files = list(MAESTRO_DIR.glob('**/*.wav'))
midi_files = list(MAESTRO_DIR.glob('**/*.midi'))
print(f'\n✅ MAESTRO v3.0.0: {len(wav_files)} WAV files, {len(midi_files)} MIDI files')

if len(wav_files) < 1200:
    print(f'⚠️ WARNING: Expected ~1276 WAV files, got {len(wav_files)}.')
    print('Dataset may be incomplete. Check disk space and re-run.')

In [ ]:
import csv

# Parse the MAESTRO metadata CSV for train/val/test splits
metadata_csv = MAESTRO_DIR / 'maestro-v3.0.0.csv'

tracks = {'train': [], 'validation': [], 'test': []}

with open(metadata_csv) as f:
    reader = csv.DictReader(f)
    for row in reader:
        split = row['split']
        audio_path = MAESTRO_DIR / row['audio_filename']
        midi_path = MAESTRO_DIR / row['midi_filename']

        if audio_path.exists() and midi_path.exists():
            tracks[split].append({
                'audio': str(audio_path),
                'midi': str(midi_path),
                'duration': float(row['duration']),
                'composer': row.get('canonical_composer', 'Unknown'),
                'title': row.get('canonical_title', 'Unknown'),
            })

for split, track_list in tracks.items():
    total_hours = sum(t['duration'] for t in track_list) / 3600
    print(f'{split:12s}: {len(track_list):4d} tracks, {total_hours:.1f} hours')

print(f'\nTotal: {sum(len(v) for v in tracks.values())} tracks')

## 2. Dataset & Preprocessing

In [ ]:
# ============================================================================
# CONFIG — must match inference in piano_transcriber.py EXACTLY
# ============================================================================

CONFIG = {
    'sample_rate': 16000,    # 16kHz (standard for piano AMT)
    'hop_length': 256,
    'n_mels': 229,           # 229 mel bins (same as Onsets & Frames)
    'n_fft': 2048,
    'fmin': 30.0,            # A0 = 27.5 Hz, with some margin
    'fmax': 8000.0,          # Covers piano range + harmonics
    'chunk_duration': 5.0,   # seconds per training chunk
    'num_keys': 88,          # A0 (21) to C8 (108) — full piano
    'min_midi': 21,          # A0
    'max_midi': 108,         # C8
    'batch_size': 12,        # Slightly smaller — piano model is heavier
    'learning_rate': 1e-4,
    'num_epochs': 50,
}

SAMPLE_RATE = CONFIG['sample_rate']
HOP_LENGTH = CONFIG['hop_length']
N_MELS = CONFIG['n_mels']
N_FFT = CONFIG['n_fft']
NUM_KEYS = CONFIG['num_keys']
MIN_MIDI = CONFIG['min_midi']
MAX_MIDI = CONFIG['max_midi']
CHUNK_DURATION = CONFIG['chunk_duration']
CHUNK_SAMPLES = int(CHUNK_DURATION * SAMPLE_RATE)  # 80000
CHUNK_FRAMES = int(CHUNK_SAMPLES / HOP_LENGTH)      # 312

print(f'Chunk: {CHUNK_DURATION}s = {CHUNK_SAMPLES} samples = {CHUNK_FRAMES} frames')
print(f'Piano range: MIDI {MIN_MIDI}-{MAX_MIDI} ({NUM_KEYS} keys)')
print(f'Mel: {N_MELS} bins, {SAMPLE_RATE} Hz')

In [ ]:
import numpy as np
import librosa
import pretty_midi
import torch
from torch.utils.data import Dataset, DataLoader
from pathlib import Path
import warnings
warnings.filterwarnings('ignore')


def midi_to_piano_targets(midi_path, total_frames, sr=SAMPLE_RATE, hop=HOP_LENGTH):
    """
    Convert MIDI file to onset/frame/velocity targets for 88-key piano.

    Returns:
        onset:    (88, total_frames) binary
        frame:    (88, total_frames) binary
        velocity: (88, total_frames) float [0-1]
    """
    onset = np.zeros((NUM_KEYS, total_frames), dtype=np.float32)
    frame = np.zeros((NUM_KEYS, total_frames), dtype=np.float32)
    velocity = np.zeros((NUM_KEYS, total_frames), dtype=np.float32)

    frames_per_sec = sr / hop

    try:
        midi = pretty_midi.PrettyMIDI(str(midi_path))
    except Exception:
        return onset, frame, velocity

    for instrument in midi.instruments:
        if instrument.is_drum:
            continue

        for note in instrument.notes:
            # Map MIDI note to piano key index (0-87)
            key_idx = note.pitch - MIN_MIDI
            if key_idx < 0 or key_idx >= NUM_KEYS:
                continue

            onset_frame = int(note.start * frames_per_sec)
            offset_frame = int(note.end * frames_per_sec)

            if onset_frame >= total_frames:
                continue

            offset_frame = min(offset_frame, total_frames - 1)
            vel = note.velocity / 127.0

            # Onset: single frame
            onset[key_idx, onset_frame] = 1.0

            # Frame: all frames while note sounds
            frame[key_idx, onset_frame:offset_frame + 1] = 1.0

            # Velocity: at onset
            velocity[key_idx, onset_frame] = vel

    return onset, frame, velocity


class PianoTranscriptionDataset(Dataset):
    """
    Dataset: yields (mel_chunk, onset_target, frame_target, velocity_target)
    from MAESTRO audio + MIDI pairs.
    """

    def __init__(self, track_list, chunks_per_track=6):
        self.tracks = track_list
        self.chunks_per_track = chunks_per_track
        self._cache = {}

    def __len__(self):
        return len(self.tracks) * self.chunks_per_track

    def _load_track(self, idx):
        track_idx = idx // self.chunks_per_track

        if track_idx in self._cache:
            return self._cache[track_idx]

        track = self.tracks[track_idx]

        # Load audio
        try:
            audio, sr = librosa.load(track['audio'], sr=SAMPLE_RATE, mono=True)
        except Exception:
            audio = np.zeros(CHUNK_SAMPLES * 2, dtype=np.float32)

        # Compute mel spectrogram
        mel = librosa.feature.melspectrogram(
            y=audio, sr=SAMPLE_RATE,
            n_fft=N_FFT, hop_length=HOP_LENGTH,
            n_mels=N_MELS, fmin=CONFIG['fmin'], fmax=CONFIG['fmax']
        )
        mel = np.log(mel + 1e-8).astype(np.float32)  # MUST match inference!

        total_frames = mel.shape[1]

        # Build targets from MIDI
        onset, frame, velocity = midi_to_piano_targets(track['midi'], total_frames)

        result = (mel, onset, frame, velocity, total_frames)

        # Cache (limit to avoid OOM — MAESTRO tracks are long)
        if len(self._cache) < 50:
            self._cache[track_idx] = result

        return result

    def __getitem__(self, idx):
        mel, onset, frame, velocity, total_frames = self._load_track(idx)

        # Random chunk
        if total_frames <= CHUNK_FRAMES:
            pad = CHUNK_FRAMES - total_frames
            mel_chunk = np.pad(mel, ((0, 0), (0, pad)))
            onset_chunk = np.pad(onset, ((0, 0), (0, pad)))
            frame_chunk = np.pad(frame, ((0, 0), (0, pad)))
            vel_chunk = np.pad(velocity, ((0, 0), (0, pad)))
        else:
            start = np.random.randint(0, total_frames - CHUNK_FRAMES)
            mel_chunk = mel[:, start:start + CHUNK_FRAMES]
            onset_chunk = onset[:, start:start + CHUNK_FRAMES]
            frame_chunk = frame[:, start:start + CHUNK_FRAMES]
            vel_chunk = velocity[:, start:start + CHUNK_FRAMES]

        return (
            torch.from_numpy(mel_chunk),
            torch.from_numpy(onset_chunk),
            torch.from_numpy(frame_chunk),
            torch.from_numpy(vel_chunk),
        )


print('✅ Dataset class defined')

In [ ]:
# Build data loaders from MAESTRO splits

train_tracks = tracks['train']
val_tracks = tracks['validation']

print(f'Train: {len(train_tracks)} tracks')
print(f'Val:   {len(val_tracks)} tracks')

train_dataset = PianoTranscriptionDataset(train_tracks, chunks_per_track=6)
val_dataset = PianoTranscriptionDataset(val_tracks, chunks_per_track=4)

train_loader = DataLoader(
    train_dataset, batch_size=CONFIG['batch_size'],
    shuffle=True, num_workers=2, pin_memory=True,
    drop_last=True
)
val_loader = DataLoader(
    val_dataset, batch_size=CONFIG['batch_size'],
    shuffle=False, num_workers=2, pin_memory=True
)

print(f'Train batches/epoch: {len(train_loader)}')
print(f'Val batches/epoch:   {len(val_loader)}')

# Sanity check
mel_b, onset_b, frame_b, vel_b = next(iter(train_loader))
print(f'\nBatch shapes:')
print(f'  Mel:      {mel_b.shape}')      # (B, 229, 312)
print(f'  Onset:    {onset_b.shape}')     # (B, 88, 312)
print(f'  Frame:    {frame_b.shape}')     # (B, 88, 312)
print(f'  Velocity: {vel_b.shape}')       # (B, 88, 312)
print(f'  Onset active: {onset_b.sum():.0f} / {onset_b.numel()}')
print(f'  Frame active: {frame_b.sum():.0f} / {frame_b.numel()}')

## 3. Model Architecture

Piano model uses a larger architecture than drums/bass since:
- 88 output keys (vs 8 drums or 96 bass string×fret)
- Highly polyphonic (10+ simultaneous notes)
- Needs velocity sensitivity for expressive dynamics
- 229 mel bins (vs 128 for drums)

In [ ]:
import torch
import torch.nn as nn


class PianoTranscriptionModel(nn.Module):
    """
    CRNN for piano transcription: Mel → Conv2D → BiLSTM → onset/frame/velocity

    Inspired by Onsets and Frames (Hawthorne et al. 2018) with:
    - Frame prediction conditioned on onset prediction
    - Velocity regression head
    - BatchNorm + Dropout for regularization

    Input:  (batch, n_mels=229, time)
    Output: onset    (batch, 88, time)  — per-key onset probability
            frame    (batch, 88, time)  — per-key frame activation
            velocity (batch, 88, time)  — velocity at onsets [0-1]
    """

    def __init__(self, n_mels=229, num_keys=88,
                 hidden_size=256, num_layers=2, dropout=0.25):
        super().__init__()

        self.num_keys = num_keys

        # CNN encoder — pools frequency only via MaxPool2d((2,1))
        # 229 → 114 → 57 → 28 → 14
        self.cnn = nn.Sequential(
            # Block 1: 229 → 114
            nn.Conv2d(1, 48, kernel_size=3, padding=1),
            nn.BatchNorm2d(48),
            nn.ReLU(),
            nn.Conv2d(48, 48, kernel_size=3, padding=1),
            nn.BatchNorm2d(48),
            nn.ReLU(),
            nn.MaxPool2d((2, 1)),  # 229 → 114
            nn.Dropout2d(dropout),

            # Block 2: 114 → 57
            nn.Conv2d(48, 64, kernel_size=3, padding=1),
            nn.BatchNorm2d(64),
            nn.ReLU(),
            nn.MaxPool2d((2, 1)),  # 114 → 57
            nn.Dropout2d(dropout),

            # Block 3: 57 → 28
            nn.Conv2d(64, 96, kernel_size=3, padding=1),
            nn.BatchNorm2d(96),
            nn.ReLU(),
            nn.MaxPool2d((2, 1)),  # 57 → 28
            nn.Dropout2d(dropout),

            # Block 4: 28 → 14
            nn.Conv2d(96, 128, kernel_size=3, padding=1),
            nn.BatchNorm2d(128),
            nn.ReLU(),
            nn.MaxPool2d((2, 1)),  # 28 → 14
            nn.Dropout2d(dropout),
        )

        # After CNN: (batch, 128, 14, time)
        cnn_output_dim = 128 * 14  # 1792

        # Onset LSTM
        self.onset_lstm = nn.LSTM(
            input_size=cnn_output_dim,
            hidden_size=hidden_size,
            num_layers=num_layers,
            batch_first=True,
            bidirectional=True,
            dropout=dropout if num_layers > 1 else 0,
        )

        # Frame LSTM (conditioned on onset predictions)
        self.frame_lstm = nn.LSTM(
            input_size=cnn_output_dim + num_keys,  # CNN features + onset_pred
            hidden_size=hidden_size,
            num_layers=num_layers,
            batch_first=True,
            bidirectional=True,
            dropout=dropout if num_layers > 1 else 0,
        )

        lstm_output_dim = hidden_size * 2  # bidirectional → 512

        # Output heads
        self.onset_head = nn.Linear(lstm_output_dim, num_keys)
        self.frame_head = nn.Linear(lstm_output_dim, num_keys)
        self.velocity_head = nn.Sequential(
            nn.Linear(lstm_output_dim, num_keys),
            nn.Sigmoid(),  # velocity is 0-1
        )

    def forward(self, x):
        """
        Args:
            x: (batch, n_mels, time) — mel spectrogram
        Returns:
            onset:    (batch, 88, time)
            frame:    (batch, 88, time)
            velocity: (batch, 88, time)
        """
        batch, n_mels, time = x.shape

        # CNN: (batch, 1, n_mels, time) → (batch, 128, 14, time)
        cnn_out = self.cnn(x.unsqueeze(1))

        # Reshape for LSTM: (batch, time, features)
        cnn_features = cnn_out.permute(0, 3, 1, 2)  # (B, T, 128, 14)
        cnn_features = cnn_features.reshape(batch, time, -1)  # (B, T, 1792)

        # Onset prediction
        onset_out, _ = self.onset_lstm(cnn_features)
        onset_logits = self.onset_head(onset_out)      # (B, T, 88)
        onset_pred = torch.sigmoid(onset_logits)

        # Frame prediction (conditioned on onset)
        frame_input = torch.cat([cnn_features, onset_pred.detach()], dim=-1)
        frame_out, _ = self.frame_lstm(frame_input)
        frame_logits = self.frame_head(frame_out)      # (B, T, 88)
        frame_pred = torch.sigmoid(frame_logits)

        # Velocity prediction
        velocity_pred = self.velocity_head(onset_out)   # (B, T, 88)

        # Transpose to (batch, keys, time)
        return (
            onset_pred.permute(0, 2, 1),
            frame_pred.permute(0, 2, 1),
            velocity_pred.permute(0, 2, 1),
        )


# Test model
model = PianoTranscriptionModel()
total_params = sum(p.numel() for p in model.parameters())
print(f'Model parameters: {total_params:,}')

# Dummy forward pass
dummy = torch.randn(2, N_MELS, CHUNK_FRAMES)
onset, frame, vel = model(dummy)
print(f'Input:    {dummy.shape}')
print(f'Onset:    {onset.shape}')    # (2, 88, 312)
print(f'Frame:    {frame.shape}')    # (2, 88, 312)
print(f'Velocity: {vel.shape}')      # (2, 88, 312)
assert onset.shape == (2, NUM_KEYS, CHUNK_FRAMES)
print('\n✅ Model architecture verified')

## 4. Training Loop

In [ ]:
import time
from tqdm import tqdm

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f'Training on: {device}')

model = PianoTranscriptionModel().to(device)
optimizer = torch.optim.AdamW(model.parameters(), lr=CONFIG['learning_rate'])
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(
    optimizer, mode='min', patience=5, factor=0.5
)

onset_loss_fn = nn.BCELoss()
frame_loss_fn = nn.BCELoss()
velocity_loss_fn = nn.MSELoss()

# Resume from checkpoint
start_epoch = 0
best_val_loss = float('inf')

checkpoint_files = sorted(Path(DRIVE_OUTPUT).glob('checkpoint_epoch_*.pt'))
best_model_path = Path(DRIVE_OUTPUT) / 'best_piano_model.pt'

if checkpoint_files:
    latest = checkpoint_files[-1]
    print(f'Resuming from {latest.name}...')
    ckpt = torch.load(latest, map_location=device, weights_only=False)
    model.load_state_dict(ckpt['model_state_dict'])
    optimizer.load_state_dict(ckpt['optimizer_state_dict'])
    start_epoch = ckpt['epoch'] + 1
    best_val_loss = ckpt.get('val_loss', float('inf'))
    print(f'Resumed at epoch {start_epoch}, best_val_loss={best_val_loss:.5f}')
elif best_model_path.exists():
    ckpt = torch.load(best_model_path, map_location=device, weights_only=False)
    model.load_state_dict(ckpt['model_state_dict'])
    optimizer.load_state_dict(ckpt['optimizer_state_dict'])
    start_epoch = ckpt['epoch'] + 1
    best_val_loss = ckpt.get('val_loss', float('inf'))
    print(f'Resumed at epoch {start_epoch}, best_val_loss={best_val_loss:.5f}')

print(f'\nStarting training from epoch {start_epoch}...')
print(f'Epochs: {CONFIG["num_epochs"]}, Batch size: {CONFIG["batch_size"]}')

In [ ]:
# ============================================================================
# TRAINING LOOP
# ============================================================================

NUM_EPOCHS = CONFIG['num_epochs']
train_losses = []
val_losses = []

for epoch in range(start_epoch, NUM_EPOCHS):
    epoch_start = time.time()

    # ---- Train ----
    model.train()
    running_loss = 0.0
    num_batches = 0

    pbar = tqdm(train_loader, desc=f'Epoch {epoch}/{NUM_EPOCHS}')
    for mel_batch, onset_tgt, frame_tgt, vel_tgt in pbar:
        mel_batch = mel_batch.to(device)
        onset_tgt = onset_tgt.to(device)
        frame_tgt = frame_tgt.to(device)
        vel_tgt = vel_tgt.to(device)

        onset_pred, frame_pred, vel_pred = model(mel_batch)

        loss_onset = onset_loss_fn(onset_pred, onset_tgt)
        loss_frame = frame_loss_fn(frame_pred, frame_tgt)

        # Velocity loss only where onsets exist
        onset_mask = onset_tgt > 0.5
        if onset_mask.any():
            loss_vel = velocity_loss_fn(
                vel_pred[onset_mask], vel_tgt[onset_mask]
            )
        else:
            loss_vel = torch.tensor(0.0, device=device)

        loss = loss_onset + loss_frame + 0.5 * loss_vel

        optimizer.zero_grad()
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
        optimizer.step()

        running_loss += loss.item()
        num_batches += 1
        pbar.set_postfix(loss=f'{loss.item():.4f}')

    train_loss = running_loss / max(num_batches, 1)
    train_losses.append(train_loss)

    # ---- Validation ----
    model.eval()
    val_running = 0.0
    val_batches = 0

    with torch.no_grad():
        for mel_batch, onset_tgt, frame_tgt, vel_tgt in val_loader:
            mel_batch = mel_batch.to(device)
            onset_tgt = onset_tgt.to(device)
            frame_tgt = frame_tgt.to(device)
            vel_tgt = vel_tgt.to(device)

            onset_pred, frame_pred, vel_pred = model(mel_batch)

            loss_onset = onset_loss_fn(onset_pred, onset_tgt)
            loss_frame = frame_loss_fn(frame_pred, frame_tgt)

            onset_mask = onset_tgt > 0.5
            if onset_mask.any():
                loss_vel = velocity_loss_fn(
                    vel_pred[onset_mask], vel_tgt[onset_mask]
                )
            else:
                loss_vel = torch.tensor(0.0, device=device)

            loss = loss_onset + loss_frame + 0.5 * loss_vel
            val_running += loss.item()
            val_batches += 1

    val_loss = val_running / max(val_batches, 1)
    val_losses.append(val_loss)

    scheduler.step(val_loss)

    elapsed = time.time() - epoch_start
    lr = optimizer.param_groups[0]['lr']

    print(f'Epoch {epoch}/{NUM_EPOCHS} | '
          f'Train: {train_loss:.4f} | Val: {val_loss:.4f} | '
          f'LR: {lr:.2e} | Time: {elapsed:.0f}s')

    # Save best model
    if val_loss < best_val_loss:
        best_val_loss = val_loss
        torch.save({
            'epoch': epoch,
            'model_state_dict': model.state_dict(),
            'optimizer_state_dict': optimizer.state_dict(),
            'val_loss': val_loss,
            'config': CONFIG,
        }, str(best_model_path))
        print(f'  ✅ Best model saved (val_loss={val_loss:.5f})')

    # Periodic checkpoint every 5 epochs
    if (epoch + 1) % 5 == 0:
        ckpt_path = Path(DRIVE_OUTPUT) / f'checkpoint_epoch_{epoch:03d}.pt'
        torch.save({
            'epoch': epoch,
            'model_state_dict': model.state_dict(),
            'optimizer_state_dict': optimizer.state_dict(),
            'val_loss': val_loss,
            'config': CONFIG,
        }, str(ckpt_path))
        print(f'  💾 Checkpoint saved: {ckpt_path.name}')

print(f'\n🎉 Training complete! Best val_loss: {best_val_loss:.5f}')
print(f'Best model saved to: {best_model_path}')

## 5. Training Curves

In [ ]:
import matplotlib.pyplot as plt

fig, ax = plt.subplots(figsize=(10, 5))
ax.plot(train_losses, label='Train Loss', color='steelblue')
ax.plot(val_losses, label='Val Loss', color='coral')
ax.set_xlabel('Epoch')
ax.set_ylabel('Loss')
ax.set_title(f'Piano Transcription Training (best val_loss={best_val_loss:.5f})')
ax.legend()
ax.grid(True, alpha=0.3)
plt.tight_layout()
plt.savefig(f'{DRIVE_OUTPUT}/training_curves.png', dpi=150)
plt.show()

## 6. Evaluation

In [ ]:
# Load best model and evaluate
ckpt = torch.load(str(best_model_path), map_location=device, weights_only=False)
model.load_state_dict(ckpt['model_state_dict'])
model.eval()

print(f'Loaded best model from epoch {ckpt["epoch"]}, val_loss={ckpt["val_loss"]:.5f}')
print(f'Config: {ckpt["config"]}')

# Quick inference test
with torch.no_grad():
    test_input = torch.randn(1, N_MELS, CHUNK_FRAMES).to(device)
    onset, frame, vel = model(test_input)
    print(f'\nInference test:')
    print(f'  Input:    {test_input.shape}')
    print(f'  Onset:    {onset.shape}, range [{onset.min():.3f}, {onset.max():.3f}]')
    print(f'  Frame:    {frame.shape}, range [{frame.min():.3f}, {frame.max():.3f}]')
    print(f'  Velocity: {vel.shape}, range [{vel.min():.3f}, {vel.max():.3f}]')

print(f'\n✅ Model ready for deployment!')
print(f'Copy {best_model_path} to backend/models/pretrained/best_piano_model.pt')